# Julia Demo

This is an adapted version of the tutorial at juliabox.com, chapter 2, DataSciences - Algorithms.

It differs from that demo in that it's 
1. shorter
2. it uses julia v1.0
3. it uses the piping syntax from the `Query.jl` package

### Example 1: Kmeans Clustering

Let's start with some data.

The Sacramento real estate transactions file that we download next is a list of 985 real estate transactions in the Sacramento area reported over a five-day period,

## Data Processing

* lets download some data and do some work on it.

In [ ]:
using DataFrames
using CSV
download("http://samplecsvs.s3.amazonaws.com/Sacramentorealestatetransactions.csv","houses.csv")
houses = CSV.read("houses.csv")

In [ ]:
using StatPlots
@df houses scatter(:sq__ft,:price,markersize=3,xlab="square feet",ylab="price")

What's with those houses at zero size and positive prices? must be wrong.

In [ ]:
using Query
# x = @from i in houses begin
#     @where i.sq__ft > 0
#     @select {i.sq__ft,i.price}
#     @collect DataFrame
# end
# @df x scatter(:sq__ft,:price,markersize=3,xlab="square feet",ylab="price")
# even better: in a pipeline!
houses |>
    @filter(_.sq__ft > 0) |>
    @df scatter(:sq__ft,:price,markersize=3,xlab="square feet",ylab="price")

## Clustering

* Let's do some clustering
* we want to cluster on location and see if we can replicate the post code groupings

In [ ]:
using Clustering

X = @from i in houses begin
    @where i.sq__ft > 0
    @select {i.latitude,i.longitude}
    @collect DataFrame
end

let's convert this into a matrix of `Float64`:

In [ ]:
y = convert(Matrix{Float64},X)

As a first pass at guessing how many clusters we might need, let's use the number of zip codes in our data.

(Try changing this to see how it impacts results!)

In [ ]:
filter_houses = houses[houses[:sq__ft].>0,:]
k = length(unique(filter_houses[:zip])) 

Now we can do kmeans on this data! Note that we need to transpose `y` so that each column is a feature of the data.

In [ ]:
yp = convert(Array{Float64},y')
C = kmeans(yp,k) # try changing k

Now let's create a new data frame, `df`, with all the same data as `filter_houses` that also includes a column for the cluster to which each house has been assigned.

In [ ]:
df = filter_houses[[:city,:latitude,:longitude,:zip]]
df[:cluster] = C.assignments
df

Let's plot each cluster as a different color.

In [ ]:
"plot the i-th cluster"
function ploti(df,i)
    df |>
        @filter(_.cluster == i) |>
        @df scatter!(:latitude,:longitude,markersize=4) # note the !
end

p_cluster = scatter(legend=false);
for i in 1:k
    p_cluster = ploti(df,i)
end
title!(p_cluster,"Houses color-coded by cluster")
p_cluster

In [ ]:
"plot houses by zip codes"
function plotz(df,i)
    df |>
        @filter(_.zip == i) |>
        @df scatter!(:latitude,:longitude)
end

unique_zips = unique(filter_houses[:zip])

p_zips = scatter(leg=false)
for uzip in unique_zips
    p_zips = plotz(filter_houses,uzip);
end
title!(p_zips,"Houses color-coded by zip code")
p_zips

How do they compare?

In [ ]:
plot(p_cluster,p_zips,layout=(2,1))

## Let's do a hand-coded linear regression shootout!

It's Python vs Julia!

In [ ]:
using Plots
xvals = repeat(1:0.5:10,inner=2)
yvals = 3 .+ xvals + 2 .* rand(length(xvals)).-1
scatter(xvals,yvals,color=:black,leg=false)

Now we want to fit a line through this. Linear Regression! Let's write a simple function in julia:

In [ ]:
using Statistics
function find_best_fit(xvals,yvals)
    meanx = mean(xvals)
    meany = mean(yvals)
    stdx = std(xvals)
    stdy = std(yvals)
    r = cor(xvals,yvals)
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
end

In [ ]:
a,b = find_best_fit(xvals,yvals)
ynew = a*xvals .+ b

In [ ]:
plot!(xvals,ynew)

now more data!

In [ ]:
xvals = 1:100000;
xvals = repeat(xvals,inner=3);
yvals = 3 .+ xvals + 2 .* rand(length(xvals)).-1;

In [ ]:
@time a,b = find_best_fit(xvals,yvals)

In [ ]:
using PyCall
using Conda

In [ ]:
py"""
import numpy
def find_best_fit_python(xvals,yvals):
    meanx = numpy.mean(xvals)
    meany = numpy.mean(yvals)
    stdx = numpy.std(xvals)
    stdy = numpy.std(yvals)
    r = numpy.corrcoef(xvals,yvals)[0][1]
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
"""

In [ ]:
find_best_fit_python = py"find_best_fit_python"

In [ ]:
xpy = PyObject(xvals)
ypy = PyObject(yvals)
@time a,b = find_best_fit_python(xpy,ypy)

In [ ]:
using BenchmarkTools

In [ ]:
@btime a,b = find_best_fit_python(xvals,yvals)

In [ ]:
@btime a,b = find_best_fit(xvals,yvals)

## How fast is the GLM package regression?

In [ ]:
using GLM
using DataFrames
df = DataFrame(x = xvals,y=yvals)
@btime glm = lm(@formula(y ~ x), df)